# 함수형 API 소개
`함수형 API`에서는 직접 텐서들의 입출력을 다룹니다. 함수처럼 층을 사용하여 텐서를 입력받고 출력합니다(그래서 함수형 API라고 부릅니다.)

In [1]:
import tensorflow as tf   #tf 2.0 GPU error 해결법
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.7)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [2]:
from tensorflow.keras import Input, layers
input_tensor = Input(shape=(32, ))
dense = layers.Dense(32, activation='relu')
output_tensor = dense(input_tensor)

Sequential 모델과 함수형 API로 만든 동일한 모델을 나란히 비교해 보겠습니다.

In [3]:
from tensorflow.keras.models import Sequential, Model

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras import Input, layers

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


Model 객체를 사용한 컴파일, 훈련, ,평가 API는 Sequential 클래스와 같습니다.

In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)
print(score)

Epoch 1/10
1000/1000 [==============================] - 0s 396us/sample - loss: 12.0406
Epoch 2/10
1000/1000 [==============================] - 0s 17us/sample - loss: 12.7532
Epoch 3/10
1000/1000 [==============================] - 0s 18us/sample - loss: 14.4239
Epoch 4/10
1000/1000 [==============================] - 0s 20us/sample - loss: 16.8564
Epoch 5/10
1000/1000 [==============================] - 0s 19us/sample - loss: 19.5777
Epoch 6/10
1000/1000 [==============================] - 0s 20us/sample - loss: 22.7254
Epoch 7/10
1000/1000 [==============================] - 0s 18us/sample - loss: 26.4460
Epoch 8/10
1000/1000 [==============================] - 0s 18us/sample - loss: 31.0271
Epoch 9/10
1000/1000 [==============================] - 0s 18us/sample - loss: 35.8535
Epoch 10/10
1000/1000 [==============================] - 0s 85us/sample - loss: 44.0642
44.064247009277345


# 다중 입력 모델
함수형 API는 다중 입력 모델을 만드는 데 사용할 수 있습니다. 일반적으로 이런 모델은 서로 다른 입력 가지를 합치기 위해 여러 텐서를 연겷할 수 있는 층을 사용합니다. 텐서를 더하거나 이어 붙이는 식입니다. 이와 관련된 케라스 함수는 `keras.layers.add`, `keras.layers.concatenate`등입니다. 아주 간단한 다중 입력 모델을 살펴보겠습니다. 질문-응답(question-answering) 모델입니다.

전형적인 질문-응답 모델은 2개의 입력을 가집니다. 하나는 자연어 질문이고, 또 하나는 답변에 필요한 정보가 담겨있는 텍스트(예를 들어 뉴스 기사)입니다. 그러면 모텔은 답을 출력해야 합니다. 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통해 한 단어로 된 답을 출력하는 것입니다.

다은은 함수형 API를 사용하여 이런 모델을 만드는 예입니다. 텍스트와 질문을 벡터로 인코딩하여 독립된 입력 2개를 정의합니다. 그다음 이 벡터를 결하고 그 위에 소프트맥스 분류기를 추가합니다.

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input, layers

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스입니다. 입력 이름을 지정할 수 있습니다.
text_input = Input(shape=(None, ), dtype='int32', name='text')
# 입력을 크기가 64인 벡터의 시퀀스로 임베딩합니다.
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
# LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩 합니다.
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(32)(embedded_question)

# 인코딩된 질문과 텍스트를 연결합니다.
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 소프트맥스 분류기를 추가합니다.
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# 모델 객체를 만들고 2개의 입력과 출력을 주입합니다.
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

W0909 19:41:26.492850 12036 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x000001ED7CC14828>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0909 19:41:26.552695 12036 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x000001ED7DFBF160>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

그럼 이렇게 입력이 2개인 모델은 어떻게 훈렪할까요? 두가지 방식이 있습니다.. 넘파이 배열의 리스트를 주입하거나 입력 이름과 넘파이 배열로 이루어진 데셔너리를 모델의 입력으로 주입할 수 있습니다. 당연하세 두번째 방식은 입력 이름을 설정했을 때 사용할 수 있습니다.

In [7]:
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

# 랜덤한 넘파이 배열을 생성합니다.
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
# 답은 정수가 아닌 원-핫 인코딩 된 벡터입니다.
answers = to_categorical(answers)

# 리스트 입력을 사용하여 학습합니다.
model.fit([text, question], answers, epochs=10, batch_size=128)
# 딕셔너리 입력을 사용하여 학습합니다(입력 이름을 지정했을 때만 사용할 수 있습니다.)
model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 6.2150 - acc: 0.0030
Epoch 2/10
1000/1000 [==============================] - 0s 100us/sample - loss: 6.1965 - acc: 0.0610
Epoch 3/10
1000/1000 [==============================] - 0s 89us/sample - loss: 6.1292 - acc: 0.0040
Epoch 4/10
1000/1000 [==============================] - 0s 90us/sample - loss: 6.0509 - acc: 0.0070
Epoch 5/10
1000/1000 [==============================] - 0s 91us/sample - loss: 5.9964 - acc: 0.0080
Epoch 6/10
1000/1000 [==============================] - 0s 91us/sample - loss: 5.8957 - acc: 0.0140
Epoch 7/10
1000/1000 [==============================] - 0s 90us/sample - loss: 5.7975 - acc: 0.0190
Epoch 8/10
1000/1000 [==============================] - 0s 90us/sample - loss: 5.6999 - acc: 0.0370
Epoch 9/10
1000/1000 [==============================] - 0s 91us/sample - loss: 5.6214 - acc: 0.0470
Epoch 10/10
1000/1000 [==============================] - 0s 90us/sample - loss: 5.5444 - acc: 0.0510

# 다중 출력 모델
같은 식으로 함수형 API를 사용하여 다중 출력(또는 다중 머리) 모델을 만들 수 있습니다. 간단한 예는 데이터에 있는 여러 속성을 동시에 예측하는 네트워크입니다. 예를 들어 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이 성별 소득 수준 등을 예측합니다.

In [8]:
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력층의 이름을 저장합니다
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

이런 모델을 훈련하려면 네트워크 출력마다 다른 손실 함수를 지정해야 합니다. 예를 들어 나이 예측은 스칼라 회귀 문제이지만 성별 예측은 이진 클래스 문제라 훈련 방식이 다릅니다. 경사 하강법은 하나의 스칼라 값을 최소화하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 합니다. 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것입니다. 케라스에서는 compile 메서드에 리스트나 딕셔너리를 사용하여 출력마다 다른 손실을 지정할 수 있습니다. 계산된 손실값은 전체 손실 하나로 더해지고 훈련 과정을 통해 최소화됩니다.

In [9]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# 위와 동일합니다(출력 층에 이름을 지정했을 때만 사용할 수 있습니다.)
model.compile(optimizer='rmsprop', loss={'age':'mse', 
                                         'income':'categorical_crossentropy', 
                                         'gender':'binary_crossentropy'})

손실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화할 것입니다. 그 결과 다른 작업들은 손해를 입습니다. 이를 해결하기 위해 손실값이 최종 손실에 기여하는 수준을 지정할 수 있습니다. 특히 손실 값의 스케일이 다를 때 유용합니다. 예를 들어 다음과 같이 가정해 보죠. 나이 회귀 작업에 사용되는 평균 제곱 오차(MSE) 손슬은 일반적으로 3~5 사잉의 값을 가집니다. 반면에 성별 분류 작업에 사용되는 크로스엔트로피 손실은 0.1 정도로 낮습니다. 이런 환경에서 손실에 균형을 맞추려면 크로스엔트로피 손실에 가중치 10을 주고 MSR 손실에 가중치 0.25를 줄 수 있습니다.

In [10]:
model.compile(optimizer='rmsprop', loss=['mse','categorical_crossentropy','binary_crossentropy'], loss_weights=[0.25, 1., 10.])

# 위와 동일합니다(출력 층에 이름을 지정했을 때만 사용할 수 있습니다.)
model.compile(optimizer='rmsprop', loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'},
                                   loss_weights={'age':0.25, 'income':1., 'gender':10.})

```python
# age_targets, income_targets, gender_targets가 넘파이 배열이라고 가정합니다.
model.fit(posts_input, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

model.fit(posts_input, {'age':age_targets, 'income':income_targets, 'gender':gender_targets}, epochs=10, batch_size=64)
```

# 층으로 구성된 비순환 유향 그래프
함수형 API를 사용하면 다중 입력이나 다중 출력 모델 뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있습니다. 케라스의 신경망은 층으로 구성된 어떤 `비순환 유향 그래프`(directed acyclic graph)도 만들 수 있습니다. 비순환이라는 것이 중요합니닫. 다시 말해 이 그래프는 원형을 띨 수 없습니다. 텐서 ㅌ가 자기 자신을 출력하는 층의 입력이 될 수 없습니다. 만들 수 있는 루프(즉 순호 연결)는 순환 층의 내부에 있는 것뿐입니다.

그래프로 구현된 몇 개의 신경망 컴포넌트가 널리 사용됩니다. 가장 유명한 2개는 인셉션 모듈과 잔차 연결입니다. 케라스에서 이 2개의 컴포넌트를 어떻게 구현하는지 살펴보겠습니다. 함수형 API를 사용하여 층의 그래프를 만드는 방법을 이해 하는 데 도움이 될 것입니다.

# 인셉션 모듈
`인셉션(Inception)`은 합성곱 신경망에서 인기있는 네트워크 구조입니다. 일찍이 너트워크 안의 네트워크(network-in-network) 구조에서 영감을 받아 2013-2014년에 크리스티안 세게디(Christian Szegedy)와 그의 구글 동료들이 만들었습니다. 나란히 분리된 가지를 따라 모듈을 쌓아 독립된 작은 네트워크처럼 구성합니다. 가장 기본적인 인셉션 모듈 형태는 3~4개의 가지를 가집니다. 1x1합성곱으로 시작해서 3x3 합성곱이 뒤따르고 마지막에 전체 출력 특성이 합쳐집니다. 이런 구성은 네트워크가 따로따로 공간 특성과 채널 방향의 특성을 학습하도록 돕습니다. 한 꺼번에 학습하는 것보다 효과가 더 높습니다. 더 복잡한 인셉션 모듈은 풀링 연산, 여러 가지 합성곱 사이즈(예를 들어 일부 가지에서는 3x3대신 5x5를 사용합니다), 공간 합성곱이 없는 가지(1x1 합성곱만 있습니다)로 구성될 수 있습니다. 다음은 함수형 API를 사용하여 `인셉션 V3(Inception V3)`모듈을 구현하는 예입니다. 이 예에서 입력 x는 4D텐서라고 가정합니다.

In [11]:
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model

x = Input(shape=(128,128, 3))
# 모든 가지는 동일한 스트라이드(2)를 사용합니다. 출력 크기를 동일하게 만들어 하나로 합치기 위해서입니다.
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

# 이 가지에서는 두번째 합성곱 층에서 스트라이드를 적용합니다.
branch_b = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b)

# 이 가지에서는 평균 풀링 층에서 스트라이드를 적용합니다.
branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x)
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
model = Model(x, output)

print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 128, 128, 128 512         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 128 512         input_3[0][0]                    
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 64, 64, 3)    0           input_3[0][0]                    
____________________________________________________________________________________________

`Note` 1x1 합성곱의 목적

이미 알고 있듯이 합성곱은 입력 텐서에서 타일 주변의 패치를 추출하고 각 패치에 동일한 연산을 수행합니다. 이 경유는 추출된 패치가 하나의 타일로 이루어졌을 때입니다. 이 합성곱 연산은 모든 타일 벡터를 하나의 Dense 층에 통과시키는 것과 동일합니다. 즉 입력 텐서의 채널 정보를 혼합한 특성을 계산합니다. 공간 방향으로는 정보를 섞지 않습니다(한번에 하나의 타일만 처리하기 때문입니다). 이런 1x1 합성곱(또는 점별 합성곱(pointwise convolution))은 인셉션 모듈의 특징입니다. 채널 방향의 특성 학습과 공간 방향의 특성 학습을 분리하는 데 도움을 줍니다. 채널이 공간 방향으로 상관관계가 크고 채널 간에는 독립적이라고 가정하면 납득할 만한 전략입니다.

인셉션 V3 전체 구조는 케라스의 keras.applications.inception_v3.InceptionV3에 준비되어 있으며, ImageNet 데이터셋에서 사전 훈련된 가중치를 포함하고 있습니다. 이와 아주 비슷한 모델인 `엑셉션(Xception)`도 케라스의 애플리케이션 모듈에 포함되어 있습니다. 엑셉션은 극단적인 인셉션(extreme inception)을 말합니다. 이 합성곱 구조는 인셉션에서 일부 영감을 얻었습니다. 채널 방향의 학습과 공간방향의 학습을 극잔적으로 분리한다는 아이디어에 착안하여 인셉션 모듈을 깊이별 분리 합성곱으로 바꿉니다. 이 합성곱은 깊이별 합성곱(depthwise convolution)(각 입력 채널에 따로따로 적용되는 공간 방향 합성곱) 다음에 점별 합성곱(1x1 합성곱)이 뒤따릅니다. 인셉션 모듈의 극한 형태로 공간 특성과 채널 방향 특성을 완전히 분리합니다. 엑셉션은 인셉션V3와 거의 동일한 개수의 모델 파라미터를 가지지만 실행 속도가 더 빠르고 ImageNet이나 다른 대규모 데이터셋에서 정확도가 더 높습니다. 이는 모델 파라미터를 더 효율적으로 사용하기 때문입니다.

# 잔차 연결
`잔차 연결`(residual connection)은 엑셉션을 포함하여 2015년 이후 등장한 많은 네트워크 구조에 있는 그래프 형태의 네트워크 컴퓨넌트입니다. 2015년 후반 ILSVRC ImageNet 경연 대회 우승 팀인 마이크로소프트의 허 등이 소개했습니다. 대규모 딥러닝 모델ㅇ에서 흔히 나타나는 두가지 문제인 그래디언트 소싥과 표현 병목(representational bottleneck)을 해결했습니다. 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 됩니다.

잔차 연결은 하위 층의 출력을 상위층의 입력으로 사용합니다. 순서대로 놓인 네트워크를 질러가는 연결이 만들어집니다. 하위 층의 출력이 상의 층의 활성화 출력에 연결괴는 것이 아니고 더해집니다. 따라서 두 출력의 크기가 동일해야 합니다. 크기가 다르면 선형변환을 사용하여 하의층의 활성화 출력을 목표 크기로 변환합니다(예를 들어 활성화 함수를 사용하지 않는 Dense층이나 합성곱의 특성 맵이라면 활성화 합수가 없는 1x1합성곱).

다음 코드는 케라스에서 특성맵의 크기가 같을 때 원본을 그대로 사용하는 잔차 연결을 구현한 예입니다. 여기서는 입력 x가 4D텐서라고 가정합니다.'

In [12]:
from tensorflow.keras import Input, layers

x = Input(shape=(128,128,128))
# x에 어떤 변환을 적용합니다.
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

# 원본 x를 출력 특성에 더합니다.
y = layers.add([y,x])

다음은 특성 맵의 크기가 다를 때 선형 변환을 사용하여 잔차 연결을 구현한 예입니다(여기에서도 입력 x가 4D텐서라고 가정합니다)

In [13]:
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model

x = Input(shape=(128,128,128))
# x에 어떤 변환을 적용합니다.
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# y와 크기를 맞추기 위해 1x1합성곱을 사용하여 원본 텐서 x를 다운샘플링합니다.
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
# 원본 x를 출력 특성에 더합니다.
y = layers.add([y,residual])

model = Model(x, y)
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 12 0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 128, 128, 128 147584      input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 128, 128, 128 147584      conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 128)  0           conv2d_11[0][0]                  
____________________________________________________________________________________________

# 층 가중치 공유
함수형 API의 중요한 또 하나의 기능은 층 객체를 여러번 재사용 할 수 있다는 것입니다. 층 객체를 두번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가중치를 재사용합니다. 이런 기능 때문에 공유 가지를 가진 모델을 만들 수 있습니다. 이런 가지는 같은 가중치를 공유하고 같은 연산을 수행합니다. 다시 말해 같은 표현을 공유하고 이런 표현을 다른 입력에서 합께 학습합니다.

다음은 케라스의 함수형 API로 공유 층(재사용 층)을 사용하는 모델을 구현하는 예입니다.

In [14]:
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model

# LSTM객체 하나를 만듭니다
lstm = layers.LSTM(32)
# 모델의 온쪽 가지를 구성합니다. 입력은 크기가 128인 벡터의 가변 길이 시퀀스입니다.
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 모델의 오른쪽 가지를 구성합니다. 기존 층 객체를 호출하면 가중치가 재사용됩니다.
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 맨 위에 분류기를 놓습니다.
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
# model.fit([left_data, right_data], targets)
print(model.summary())

W0909 19:41:34.155368 12036 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x000001EC5EBDFB70>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
unified_lstm_2 (UnifiedLSTM)    (None, 32)           20608       input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 64)           0           unified_lstm_2[0][0]       

# 층과 모델
함수형 API에서는 모델을 층처럼 사용할 수 있습니다. 모델을 '커다란 층'으로 생각해도 됩니다. Sequential 클래스와 Model 클래스에서 모두 동일합니다. 이 말은 입력 텐서로 모델을 호출해서 출력 텐서를 얻을 수 있다는 뜻입니다.
```python
y = model(x)
```
모델에서 입력 텐서와 출력 덴서가 여러 개이면 텐서의 리스트로 호출합니다.
```python
y1, y2 = model([x1,x2])
```
모델 객체를 호출할 때 모델의 가중치가 재사용됩니다. 층 객체를 호출할 떄와 정확히 같습니다. 층 객체나 모델 객체나 객체를 호풀하는 것은 항상 그 객체가 가진 학습된 표현을 재사용 합니다.

모델 객체를 재사용하는 간단한 실전 예는 듀얼 카메라에서 입력을 받는 비전 모델입니다. 두 카메라가 멫 센티미터(1인치) 간격을 두고 나란히 있습니다. 이런 모델은 깊이를 감지할 수 있습니다. 많은 애플리케이션에서 유용한 기능입니다. 왼쪽 카메라와 오른쪽 카메라에서 시각적 특징을 추출하여 합칙기 위해 2개의 독립된 모델을 사용할 필요가 없습니다. 두 입력에 저수준 처리과정이 공유될 수 있습니다. 다시 말해 가중치가 같고 동일한 표현을 공유하는 층을 사용합니다. 다음은 케라스에서 샴 비전 모델(공유 합성곱 기반 층)을 구현하는 예입니다.

In [15]:
from tensorflow.keras import Input, layers
from tensorflow.keras import applications

# 이미지 처리 기본 모델은 엑셉션 네트워크입니다.(합성곱 기반 층만 사용합니다.)
xception_base = applications.Xception(weights=None, include_top=False)
# 입력은 250x250 RGB 이미지 입니다.
left_input = Input(shape=(250,250,3))
right_input = Input(shape=(250,250,3))

# 같은 비전 모델을 두번 호출합니다.
left_features = xception_base(left_input)
right_input = xception_base(right_input)

# 합쳐진 특성은 오른쪽 입력과 왼쪽 입력에서 얻은 정보를 담고 있습니다.
merged_features = layers.concatenate([left_features, right_input], axis=-1)

이것으로 케라스의 함수형 API 소개를 마칩니다. 이는 고급 심층 신경망 구조를 구축하기 위해 필수적인 도구입니다. 여기에서 다음 내용을 배웠습니다.
* 차례대로 층을 쌓는 것 이상이 필요할 때는 Sequential API를 사용하지 않습니다.
* 함수형 API를 사용하여 다중 입력, 다중 출력, 복잡한 네트워크 토폴로지를 갖는 케라스 모델을 만드는 방법
* 다른 네트워크 가지에서 같은 층이나 모델 객체를 여러번 호출하여 가중치를 재사용하는 방법